In [ ]:
import pennylane as qml
import matplotlib.pyplot as plt
import torch
import pandas as pd
import math
import datetime

torch.manual_seed(42)
torch.set_num_threads(30)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Define QPINN
def circuit(x, basis):
    # Embedding
    for i in range(N_QUBITS):
        qml.RY(basis[i] * x, wires=i)
    
    # Variational ansatz
    for i in range(N_LAYERS):
        for j in range(N_QUBITS):
            qml.RX(theta[i,j,0], wires=j)
            qml.RY(theta[i,j,1], wires=j)
            qml.RZ(theta[i,j,2], wires=j)
    
        for j in range(N_WIRES - 1):
            qml.CNOT(wires=[j, j + 1])

    ## Cost Function
    return qml.expval(qml.sum(*[qml.PauliZ(i) for i in range(N_QUBITS)]))

class FNNBasisNet(torch.nn.Module):
    def __init__(self, n_hidden_layers, branch_width):
        super().__init__()

        self.n_hidden_layers = n_hidden_layers
        self.branch_width = branch_width
        self.layers = torch.nn.ModuleList()
        self.layers.append(torch.nn.Linear(1, branch_width))
        for i in range(n_hidden_layers -1):
            self.layers.append(torch.nn.Linear(branch_width, branch_width))
        self.layers.append(torch.nn.Linear(branch_width, N_QUBITS))
    
    def forward(self, x):
        for i in range(self.n_hidden_layers):
            x = torch.tanh(self.layers[i](x))
        x = self.layers[self.n_hidden_layers](x)
        return x

def model(x):

    # Rescale input to [-0.95, 0.95]       
    x_rescaled = 0.95 * 2*x - 0.95
    
    return qnode(x_rescaled.T, basisNet(x_rescaled.unsqueeze(1)).T)

def loss_diff_fnc():
    x = torch.linspace(0.0, 1.0, 100, requires_grad=True, device=device)
    u = model(x) 
    du_dx = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    res = du_dx - (4*u - 6*u**2 + torch.sin(50*x) + u*torch.cos(25*x) - 0.5)

    return torch.mean(res**2)

def loss_boundary_fnc():
    x = torch.linspace(0.0, 1.0, 100, requires_grad=True, device=device)
    u_0 = model(torch.zeros_like(x))
    return torch.mean((u_0 - 0.75)**2)

def loss_fnc():

    loss_diff     = loss_diff_fnc()
    loss_boundary = loss_boundary_fnc()

    return 10E1*loss_boundary + loss_diff        


def compute_loss(theta, basisNet):
    
    opt = torch.optim.LBFGS([theta, *basisNet.parameters()], line_search_fn="strong_wolfe")
    qnode = qml.QNode(circuit, dev)

    loss_history = []
    
    def closure():
        opt.zero_grad()
        l = loss_fnc()
        l.backward()
        return l
    
    start = datetime.datetime.now()
    for i in range(50):
            opt.step(closure)
            loss_history.append(loss_fnc().item())
    print(f"\t Time: {datetime.datetime.now() - start}")

    return loss_fnc().item(), loss_history


In [ ]:

qubits = [2,4,8]
layer = [4,8,12]
n_runs = 3

methods = ["Random", "IdentityBlocks", "OneBlock","Xavier Uniform", "Xavier Normal"]
data = []

for N_QUBITS in qubits:
    for N_LAYERS in layer:

        print(f"Qubits: {N_QUBITS}, Layers: {N_LAYERS}")
        dev = qml.device("default.qubit", wires=N_QUBITS)
        qnode = qml.QNode(circuit, dev)
        
        for i in range(n_runs):
            print(f"Trial {i}")
            
            # Random Parameter Initialization
            if "Random" in methods:
                basisNet = FNNBasisNet(1, 5).to(device)
                theta = 2*torch.pi*torch.rand((N_LAYERS, N_QUBITS, 3),device=device)
                theta.requires_grad = True
                loss_value, loss_history = compute_loss(theta, basisNet)
                for m, loss in enumerate(loss_history):
                    data.append({"qubits": N_QUBITS,
                                "layers": N_LAYERS,
                                "method": "Random",
                                "loss": loss,
                                "run": i,
                                "iteration": m,
                                "loss": loss})
            

            # Compute identity block initialization
            if "IdentityBlocks" in methods:
                basisNet = FNNBasisNet(1, 5).to(device)
                theta = 2*torch.pi*torch.rand(N_LAYERS, N_QUBITS, 3, device=device)
                for j in range(N_LAYERS//2):
                    theta[2*j,:,:] = -theta[2*j+1,:,:]
                theta.requires_grad = True
                loss_value, loss_history = compute_loss(theta, basisNet)
                for m, loss in enumerate(loss_history):
                    data.append({"qubits": N_QUBITS,
                                "layers": N_LAYERS,
                                "method": "IdentityBlocks",
                                "loss": loss,
                                "run": i,
                                "iteration": m,
                                "loss": loss})
            

            # Compute identity block initialization
            if "OneBlock" in methods:
                basisNet = FNNBasisNet(1, 5).to(device)
                theta = 2*torch.pi*torch.rand(N_LAYERS, N_QUBITS, 3, device=device)
                for j in range(N_LAYERS//2):
                    theta[j,:,:] = -theta[N_LAYERS-j-1,:,:]
                theta.requires_grad = True
                loss_value, loss_history = compute_loss(theta, basisNet)
                for m, loss in enumerate(loss_history):
                    data.append({"qubits": N_QUBITS,
                                "layers": N_LAYERS,
                                "method": "One IdentityBlock",
                                "loss": loss,
                                "run": i,
                                "iteration": m,
                                "loss": loss})
                    

            # Xavier Uniform Initialization
            if "Xavier Uniform" in methods:
                basisNet = FNNBasisNet(1, 5).to(device)
                bound = math.sqrt(6/(2*N_QUBITS))
                theta = torch.distributions.uniform.Uniform(-bound,bound).sample([N_LAYERS,N_QUBITS,3]).to(device)
                theta.requires_grad = True
                loss_value, loss_history = compute_loss(theta, basisNet)
                for m, loss in enumerate(loss_history):
                    data.append({"qubits": N_QUBITS,
                                "layers": N_LAYERS,
                                "method": "Xavier Uniform",
                                "loss": loss,
                                "run": i,
                                "iteration": m,
                                "loss": loss})
            

            # Xavier Normal Initialization
            if "Xavier Normal" in methods:
                basisNet = FNNBasisNet(1, 5).to(device)
                variance = 1/(2*N_QUBITS)
                theta = torch.distributions.normal.Normal(0, variance).sample([N_LAYERS,N_QUBITS,3]).to(device)
                theta.requires_grad = True
                loss_value, loss_history = compute_loss(theta, basisNet)
                for m, loss in enumerate(loss_history):
                    data.append({"qubits": N_QUBITS,
                                "layers": N_LAYERS,
                                "method": "Xavier Normal",
                                "loss": loss,
                                "run": i,
                                "iteration": m,
                                "loss": loss})

df = pd.DataFrame(data)
# Save the data frame to a CSV file
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
df.to_csv(f'init_methods_{timestamp}.csv', index=False)

In [6]:
df = pd.DataFrame(data)
# Save the data frame to a CSV file
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
df.to_csv(f'init_methods_{timestamp}.csv', index=False)